In [1]:
"""
IMPORTANT: This data augmentation framework is applied with the assumption that there
are only 6 base data augmentations. If this number changes, the framework has to be modified accordingly!

"""

# Load the necessary packages:
import itertools
from pathlib import Path
from pprint import pprint
import os
import random
from matplotlib import pyplot as plt, cm
import numpy as np
import pandas as pd
# from pandas_path import path
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import minmax_scale
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score
from tqdm import tqdm

# Set the home directory:
home_dir = '/home/jupyter'

# Define the maximum number of columns in the dataframes that will be used:
pd.set_option("max_colwidth", 80)

# Set the random seed for reproducibility:
RANDOM_SEED = 42


# Functions to preprocess data signals:
def drop_frac_and_He(df):
    """
    Drops fractional m/z values, m/z values > 100, and carrier gas m/z

    Args:
        df: a dataframe representing a single sample, containing m/z values

    Returns:
        The dataframe without fractional an carrier gas m/z
    """

    # drop fractional m/z values
    df = df[df["m/z"].transform(round) == df["m/z"]]
    assert df["m/z"].apply(float.is_integer).all(), "not all m/z are integers"

    # drop m/z values greater than 99
    # df = df[df["m/z"] < 100]  # default
    
    # # drop m/z values greater than 150
    df = df[df["m/z"] < 151]

    # drop carrier gas
    df = df[df["m/z"] != 4]  # default
    
    # # drop m/z values less than 10
    # df = df[df["m/z"] >= 4]     # creates trouble!

    return df

def remove_background_abundance_pds(df):
    """
    A dummy function for convenience
    
    """
    # Modified:
    df["abun_minsub"] = df["abundance"]

    return df

# Combine all the preprocessing functions:
def preprocess_sample(df):
    df = drop_frac_and_He(df)
    df = remove_background_abundance_pds(df)
    return df


"""
    Functions to generate artificial spectra:
    Contains 6 base data augmentations
    Dervied from the script: generateArtificialSpectra.py

    Written by Sabrina Do
    NASA GSFC Summer Internship 2023
    
    Modified by Pugazhenthi Sivasankar,
    FDL 2024, SETI Institute, CA.

"""

def shift(input_array, degree_shift):
    """
    input:
        * input_array: 1D array
        * degree_shift: int, (number between +/- 3), if +, shifts to the right
    output: 
        * 1D array that is shifted
    """
    degree_shift = min(degree_shift, 3)
    degree_shift = max(degree_shift, -3)
    
    return np.roll(input_array, degree_shift)

def shiftRandom(input_array, degree_shift, percent_random_peaks_to_shift):
    """
    input:
        * input_array: 1D array
        * degree_shift: int, (number between +/- 3), if +, shifts to the right
    output: 
        * 1D array that is shifted
    """

    #### Victoria's notes ####
    # for this shiftRandom we want to shift some randomly chosen peaks. not all the peaks will be shifted.
    # We will have a parameter (percent_random_peaks_to_shift) that will represent the percentage of peaks to shift
    # these peaks will be randomly selected and they will be shifted by "degree_shift" (maximum is 3)
    # The shift should be always done from the original data (the true data) and not from previously generated data

    degree_shift = min(degree_shift, 3)
    degree_shift = max(degree_shift, -3)

    peak_indices = np.nonzero(input_array)[0]

    num_peaks_shift = int(len(peak_indices) * (percent_random_peaks_to_shift/100))

    indicesToShift = np.random.choice(peak_indices, size=num_peaks_shift, replace=False)

    shifted_array = input_array.copy()
    array_length = len(shifted_array)

    for index in indicesToShift:
        new_index = (index + degree_shift) % array_length
        shifted_array[new_index] = input_array[index]
        shifted_array[index] = 0

    
    return shifted_array

def randomizeIntensity(input_array, percentage_intensity):
    """
    input:
        * input_array: 1D array
        * percentage_intensity: int of the percentage to increase or decrease
    output:
        * 1D array with intensity increased
    """
    # probability = 0.5   # Default
    probability = 0.7
    for i in range(len(input_array)):
        if random.random() < probability:
            random_factor = random.uniform(0, percentage_intensity)
            input_array[i] *= (percentage_intensity / 100 + 1)   # Default
            # input_array[i] *= (random_factor / 100 + 1)
            
    return input_array

def add_noise_arushi(input_array, noise_level, type):
    '''
    Function that adds gaussian or uniform noise to the data. 
    '''
    max_value = max(input_array)
    len_noise = len(input_array)

    if (type == 'gaussian'):
        noise_function = np.array([random.gauss(0, max_value) for i in range(len_noise)])*noise_level
    
    elif (type == 'white'):
        noise_function = np.array([random.uniform(-max_value, max_value) for i in range(len_noise)])*noise_level
    
    # Include the generated noise:
    modified_amp = input_array + noise_function
    
    return modified_amp


def stretch2(input_array, value_stretch_param):
    """
    input:
        * input_array: 1D array
        * value_stretch_param: how much you want it to be stretched
    output:
        * 1D array
    
    function: 
        * keeps highest peak, stretches on each side of the peak
        
    Author:
        * Pugazhenthi Sivasankar, FDL 2024, SETI Institute, CA
    """
    
    # Finding the index of the peak value:
    highest_val_index = np.argmax(input_array)
    
    # Define the percentage by which the segments to the left and right side of the peak
    # will increase in value:
    percent_increase = 30
    
    # Check the locations of the highest value index and perform the stretching accordingly:
    if (highest_val_index > value_stretch_param) and (highest_val_index < (len(input_array) - value_stretch_param)):


        # Increment the left segment:
        stretched_values1 = (1 + percent_increase/100)*input_array[:highest_val_index - value_stretch_param]

        # Increment the right segment:
        stretched_values2 = (1 + percent_increase/100)*input_array[highest_val_index + value_stretch_param:]

        # Define the x and y values of the segment to be stretched:
        y_exp = np.array([stretched_values1[-1], input_array[highest_val_index], stretched_values2[0]])
        x_exp = np.array([highest_val_index - value_stretch_param, highest_val_index,
                          highest_val_index + value_stretch_param])

        # Stretching the peak to the left using a straight line:
        y_inter1 = ((y_exp[1] - y_exp[0])/(x_exp[1] - 
                                           x_exp[0]))*np.linspace(1, value_stretch_param-1, num=value_stretch_param-1)

        # Stretching the peak to the right using a straight line:
        y_inter2 = y_exp[1] + (((y_exp[2] - 
                                 y_exp[1])/(x_exp[2] - x_exp[1]))*
                               np.linspace(1, value_stretch_param, num=value_stretch_param))

        # Combining all the segments:
        stretched_comb = np.append(stretched_values1, y_inter1)
        peak_val = np.array(input_array[highest_val_index])
        # print('Peak value ' + str(peak_val))
        stretched_comb = np.append(stretched_comb, peak_val)
        stretched_comb = np.append(stretched_comb, y_inter2)
        stretched_comb = np.append(stretched_comb, stretched_values2)

    elif (highest_val_index < value_stretch_param):

        # Increment the right side:
        stretched_right = (1 + percent_increase/100)*input_array[value_stretch_param:]

        # Define the x and y values of the segment to be stretched:
        # Push the highest value to the start of the array and do a right stretch:
        y_exp = np.array([input_array[highest_val_index], stretched_right[0]])
        x_exp = np.array([0, value_stretch_param])

        # Stretching the peak to the right using a straight line:
        y_inter_right = y_exp[0] + (((y_exp[1] - 
                                 y_exp[0])/(x_exp[1] - x_exp[0]))*
                               np.linspace(0, value_stretch_param, num=value_stretch_param))    

        # Combining all the segments:
        stretched_comb = np.append(y_inter_right, stretched_right)

    else:

        # Increment the left side:
        stretched_left = (1 + percent_increase/100)*input_array[:len(input_array) - value_stretch_param]

        # Define the x and y values of the segment to be stretched:
        # Push the highest value to the end of the array and do a left stretch:
        y_exp = np.array([stretched_left[-1], input_array[highest_val_index]])
        x_exp = np.array([0, value_stretch_param])  

        # Stretching the peak to the left using a straight line:
        y_inter_left = y_exp[0] + (((y_exp[1] - 
                                 y_exp[0])/(x_exp[1] - x_exp[0]))*
                               np.linspace(0, value_stretch_param, num=value_stretch_param))    

        # Combining all the segments:
        stretched_comb = np.append(stretched_left, y_inter_left)    


    # if len(stretched_comb) ==  len(input_array):
    #     print('Stretched array and input array are equal in length')
    # else:
    #     print('Unequal array lengths for stretched and input array')

    # Return the stretched array:
    return stretched_comb


def bda_pds_each_spectra_final_norm(df, aug_param, idx):
    """
    Apply 6 base data augmentations to one time slice at a time, and then normalize the
    entire array.

    Args:
        df: dataframe with preprocessed abundance signals
        aug_param: a dictionary of data augmentation parameters
        idx_total: a list of indices at which the spectra starts in the sample dataframe

    Returns:
        dataframe with 7 additional columns (1 normalized preprocessed abundance signal
        + 6 base augmentations of the preprocessed signal)
    """

    # Initialize a counter to keep track of the number of time slices in a sample:
    time_slice_counter = 0

    # Initialize empty arrays for concatenation:
    y2_norm = np.array([])
    y2_shift = np.array([])
    y2_shiftRan = np.array([])
    y2_randInten = np.array([])
    y2_aw_noise = np.array([])
    y2_ag_noise = np.array([])
    # y2_stretch1 = np.array([])
    y2_stretch2 = np.array([])

    # Loop over the unique time slices:
    for i in range(len(idx) - 1): # To be modified

        # Extract the preprocessed signal:
        y1 = df[idx[i]:idx[i+1]]['abun_minsub']   # Change 

        # Concatenate the preprocessed signal:
        y2_norm = np.concatenate((y2_norm, y1.values))

        # Shift and concatenate the preprocessed signal:
        temp_shift = abs(shift(y1.values, aug_param["degree_shift"]))
        y2_shift = np.concatenate((y2_shift, temp_shift))

        # Randomly shift and concatenate the preprocessed signal:
        temp_rand = abs(shiftRandom(y1.values, aug_param["degree_shift"],
                                    aug_param["percent_random_peaks_to_shift"]))  
        y2_shiftRan = np.concatenate((y2_shiftRan, temp_rand))

        # Randomize the intensity and concatenate the preprocessed signal:
        temp_randInten = abs(randomizeIntensity(y1.values, aug_param["percentage_intensity"]))
        y2_randInten = np.concatenate((y2_randInten, temp_randInten))

        # Add white noise and concatenate the preprocessed signal: 
        temp_aw_noise = abs(add_noise_arushi(y1.values, aug_param["arushi_noise_level"], 'white'))
        y2_aw_noise = np.concatenate((y2_aw_noise, temp_aw_noise))  

        # Add Gaussian noise and concatenate the preprocessed signal:
        temp_ag_noise = abs(add_noise_arushi(y1.values, aug_param["arushi_noise_level"], 'gaussian'))
        y2_ag_noise = np.concatenate((y2_ag_noise, temp_ag_noise))    

        # Type 2 stretch:
        temp_stretch2 = abs(stretch2(y1.values, aug_param["value_stretch_param"]))        
        y2_stretch2 = np.concatenate((y2_stretch2, temp_stretch2))          

        time_slice_counter += 1

    # print(time_slice_counter)

    df_aug = df
    df_aug["abun_ms_norm"] = y2_norm/max(y2_norm)
    df_aug["shift"] = y2_shift/max(y2_shift)
    df_aug["shiftRan"] = y2_shiftRan/max(y2_shiftRan)
    df_aug["randInten"] = y2_randInten/max(y2_randInten)
    df_aug["aw_noise"] = y2_aw_noise/max(y2_aw_noise)
    df_aug["ag_noise"] = y2_ag_noise/max(y2_ag_noise)    
    df_aug["stretch2"] = y2_stretch2/max(y2_stretch2)

    return df_aug

def get_total_comb_aug_type(data_aug_base_type):
    
    # A function that returns a list of combinatorial data augmentation column numbers in the master dataframe.
    # Input: a dictionary of the base data augmentation types.
    # Output: a list of combinatorial data augmentation columns.

    # Function which returns subset or r length from n
    from itertools import combinations

    # Import math Library
    import math 

    def rSubset(arr, r):

        # return list of all subsets of length r
        # to deal with duplicate subsets use
        # set(list(combinations(arr, r)))
        return list(combinations(arr, r))


    # Set the number of base data augmentation types:
    bda_num = list(data_aug_base_type.keys())

    # Number of randomly selected base augmentations:
    base_aug_num = [i for i in range(2, len(data_aug_base_type)+1)]

    # Initialize a variable to count the number of combinations:
    total_comb = 0

    # Initialize an empty list:
    final_comb_aug_type = []

    # Loop over the base data augmentations:
    for i in range(len(base_aug_num)):

        # Create the combinations:
        comb_aug_type = rSubset(bda_num, base_aug_num[i])

        # Perform sanity check:
        if len(comb_aug_type) == math.comb(len(bda_num), base_aug_num[i]):
            print('Number of possible ' + str(base_aug_num[i])
                  + ' combinations out of 6 base data augmentations: ' + str(len(comb_aug_type)))

        # Horizontally concatenate the combinatorial data augmentations:
        final_comb_aug_type = final_comb_aug_type + comb_aug_type

        # Increment the counter for combinations:
        total_comb += len(comb_aug_type)

    # Print the value for verification:
    print('Total number of combinatorial augmentations: ' + str(total_comb))
    
    # Return the list of combinatorial data augmentations:
    return final_comb_aug_type

In [2]:
# Make a dictionary for the 6 base types of data augmentation:
data_aug_base_type = {
    7: "shift",
    8: "shiftRan",
    9: "randInten",
    10: "aw_noise",
    11: "ag_noise",
    12: "stretch2"
}

# Obtain the various combinations of 6 base data augmentations:
final_comb_aug_type = get_total_comb_aug_type(data_aug_base_type)

# Print the various combinations of 6 base data augmentations:
print(final_comb_aug_type)

Number of possible 2 combinations out of 6 base data augmentations: 15
Number of possible 3 combinations out of 6 base data augmentations: 20
Number of possible 4 combinations out of 6 base data augmentations: 15
Number of possible 5 combinations out of 6 base data augmentations: 6
Number of possible 6 combinations out of 6 base data augmentations: 1
Total number of combinatorial augmentations: 57
[(7, 8), (7, 9), (7, 10), (7, 11), (7, 12), (8, 9), (8, 10), (8, 11), (8, 12), (9, 10), (9, 11), (9, 12), (10, 11), (10, 12), (11, 12), (7, 8, 9), (7, 8, 10), (7, 8, 11), (7, 8, 12), (7, 9, 10), (7, 9, 11), (7, 9, 12), (7, 10, 11), (7, 10, 12), (7, 11, 12), (8, 9, 10), (8, 9, 11), (8, 9, 12), (8, 10, 11), (8, 10, 12), (8, 11, 12), (9, 10, 11), (9, 10, 12), (9, 11, 12), (10, 11, 12), (7, 8, 9, 10), (7, 8, 9, 11), (7, 8, 9, 12), (7, 8, 10, 11), (7, 8, 10, 12), (7, 8, 11, 12), (7, 9, 10, 11), (7, 9, 10, 12), (7, 9, 11, 12), (7, 10, 11, 12), (8, 9, 10, 11), (8, 9, 10, 12), (8, 9, 11, 12), (8, 10,

In [3]:
def get_comb_data_aug(df_base_aug, final_comb_aug_type):
    
    # A function that performs combinatorial data augmentations using the 6 base data augmentations.
    # NOTE: IF THE NUMBER OF BASE DATA AUGMENTATIONS IS CHANGED FROM 6 TO ANY OTHER NUMBER, THIS
    # FUNCTION SHOULD BE MODIFIED ACCORDINGLY!
    
    # Inputs:
    # 1) df_base_aug: a data frame that has the preprocessed and normalized raw signal along with
    # the 6 base data augmentations.
    # 2) final_comb_aug_type: a list of tuples that contains the combinations of the 6 base augmented 
    # data columns.
    
    # Outputs:
    # A dictionary that contains 64 data frames in the following format:
    # 1st data frame: the preprocessed and normalized raw signal
    # 2nd - 7th data frame: the 6 base data augmentations
    # 8th - 64th data frame: 57 combinatorial data augmentations.

    # Display the first few lines of the desired columns of the DataFrame:
    col_interest =  [i for i in range(6, len(df_base_aug.columns))]

    # Obtain the eid by splitting the file name:
    eid = df_base_aug["EID"][0].split(".")[0]

    # Get the total number of data augmentations including the 
    # preprocessed and normalized raw signal:
    total_aug_num = 1 + (len(df_base_aug.columns) - 7) + len(final_comb_aug_type)

    # Initialize an empty dictionary for collection:
    df_coll = {}

    # Loop over the total number of augmentations:
    for i in range(0, total_aug_num):
        if i == 0:

            # Store the preprocessed and normalized raw signal:
            df_coll["df_"+ eid + "_" + "norm"] = df_base_aug.iloc[:,[0,1,2,3,4,5,col_interest[i]]]

        elif i in [1,2,3,4,5,6]:

            # Store the base data augmentations:
            df_coll["df_"+ eid + "_" + "base{0}".format(i)] = df_base_aug.iloc[:,[0,1,2,3,4,5,col_interest[i]]]

        else:

            # Perform the combinatorial data augmentations:
            # Create a dummy dataframe and assign the first 5 columns of the master dataframe:
            dummy = df_base_aug.iloc[:,[0,1,2,3,4,5]]

            # All possible 2 combinations out of 6 base data augmentations:
            if len(final_comb_aug_type[i-7]) == 2:

                # Extract the base data augmentations:
                y1 = df_base_aug.iloc[:,final_comb_aug_type[i-7][0]]
                y2 = df_base_aug.iloc[:,final_comb_aug_type[i-7][1]]

                # Create the field name:
                combo_name = 'combo_' + \
                str(final_comb_aug_type[i-7][0]) + '_' + \
                str(final_comb_aug_type[i-7][1])

                # Find the average of the base data augmentations:
                combo_val = np.mean(np.array([y1.values,
                                              y2.values]), axis=0)


            # All possible 3 combinations out of 6 base data augmentations:
            elif len(final_comb_aug_type[i-7]) == 3:

                # Extract the base data augmentations:
                y1 = df_base_aug.iloc[:,final_comb_aug_type[i-7][0]]
                y2 = df_base_aug.iloc[:,final_comb_aug_type[i-7][1]]
                y3 = df_base_aug.iloc[:,final_comb_aug_type[i-7][2]]

                # Create the field name:
                combo_name = 'combo_' + \
                str(final_comb_aug_type[i-7][0]) + '_' + \
                str(final_comb_aug_type[i-7][1]) + '_' + \
                str(final_comb_aug_type[i-7][2])

                # Find the average of the base data augmentations:
                combo_val = np.mean(np.array([y1.values,
                                              y2.values,
                                              y3.values]), axis=0)

            # All possible 4 combinations out of 6 base data augmentations:
            elif len(final_comb_aug_type[i-7]) == 4:

                # Extract the base data augmentations:
                y1 = df_base_aug.iloc[:,final_comb_aug_type[i-7][0]]
                y2 = df_base_aug.iloc[:,final_comb_aug_type[i-7][1]]
                y3 = df_base_aug.iloc[:,final_comb_aug_type[i-7][2]]
                y4 = df_base_aug.iloc[:,final_comb_aug_type[i-7][3]]

                # Create the field name:
                combo_name = 'combo_' + \
                str(final_comb_aug_type[i-7][0]) + '_' + \
                str(final_comb_aug_type[i-7][1]) + '_' + \
                str(final_comb_aug_type[i-7][2]) + '_' + \
                str(final_comb_aug_type[i-7][3])

                # Find the average of the base data augmentations:
                combo_val = np.mean(np.array([y1.values,
                                              y2.values,
                                              y3.values,
                                              y4.values]), axis=0)            


            # All possible 5 combinations out of 6 base data augmentations:
            elif len(final_comb_aug_type[i-7]) == 5:

                # Extract the base data augmentations:
                y1 = df_base_aug.iloc[:,final_comb_aug_type[i-7][0]]
                y2 = df_base_aug.iloc[:,final_comb_aug_type[i-7][1]]
                y3 = df_base_aug.iloc[:,final_comb_aug_type[i-7][2]]
                y4 = df_base_aug.iloc[:,final_comb_aug_type[i-7][3]]
                y5 = df_base_aug.iloc[:,final_comb_aug_type[i-7][4]]

                # Create the field name:
                combo_name = 'combo_' + \
                str(final_comb_aug_type[i-7][0]) + '_' + \
                str(final_comb_aug_type[i-7][1]) + '_' + \
                str(final_comb_aug_type[i-7][2]) + '_' + \
                str(final_comb_aug_type[i-7][3]) + '_' + \
                str(final_comb_aug_type[i-7][4])

                # Find the average of the base data augmentations:
                combo_val = np.mean(np.array([y1.values,
                                              y2.values,
                                              y3.values,
                                              y4.values,
                                              y5.values]), axis=0)  

            # All possible 6 combinations out of 6 base data augmentations:
            elif len(final_comb_aug_type[i-7]) == 6:

                # Extract the base data augmentations:
                y1 = df_base_aug.iloc[:,final_comb_aug_type[i-7][0]]
                y2 = df_base_aug.iloc[:,final_comb_aug_type[i-7][1]]
                y3 = df_base_aug.iloc[:,final_comb_aug_type[i-7][2]]
                y4 = df_base_aug.iloc[:,final_comb_aug_type[i-7][3]]
                y5 = df_base_aug.iloc[:,final_comb_aug_type[i-7][4]]
                y6 = df_base_aug.iloc[:,final_comb_aug_type[i-7][5]]

                # Create the field name:
                combo_name = 'combo_' + \
                str(final_comb_aug_type[i-7][0]) + '_' + \
                str(final_comb_aug_type[i-7][1]) + '_' + \
                str(final_comb_aug_type[i-7][2]) + '_' + \
                str(final_comb_aug_type[i-7][3]) + '_' + \
                str(final_comb_aug_type[i-7][4]) + '_' + \
                str(final_comb_aug_type[i-7][5])

                # Find the average of the base data augmentations:
                combo_val = np.mean(np.array([y1.values,
                                              y2.values,
                                              y3.values,
                                              y4.values,
                                              y5.values,
                                              y6.values]), axis=0)
                
            # Normalize the combinatorial data augmentation:
            combo_val_norm = combo_val/max(combo_val)

            # Insert the averaged value and the field name as the last column of the dummy dataframe:
            dummy.insert(len(dummy.columns), combo_name, combo_val_norm)

            # Add the dummy column to dictionary collection:
            df_coll["df_"+ eid + "_" + "combo{0}".format(i-6)] = dummy
    
    # Return the dictionary of collected data frames:
    return df_coll


def display_data_aug_pds(df_coll, idx):
    
    # This function displays a randomly chosen data augmentation at 3 consecutive time slices:

    # Input:
    # df_coll: A dictionary that contains 64 data frames in the following format:
    # 1st data frame: the preprocessed and normalized raw signal
    # 2nd - 7th data frame: the 6 base data augmentations
    # 8th - 64th data frame: 57 combinatorial data augmentations. 
    
    # Output:
    # A figure with 1 x 3 subplots.

    # Get the preprocessed and normalized raw signal:
    # Ref: https://www.tutorialspoint.com/python-program-to-get-first-and-last-element-from-a-dictionary
    norm_key = list(df_coll)[0]
    df_norm = df_coll[norm_key]
    print(norm_key)
    # print(df_norm)

    # Get a randomly augmented data:
    n_rand = random.randint(1, len(df_coll) - 1)
    aug_key_rand = list(df_coll)[n_rand]
    df_aug = df_coll[aug_key_rand]
    print(aug_key_rand)
    # print(df_aug)

    # Obtain the EID for display purposes:
    chosen_eid = norm_key.split("_")[1]
    print(chosen_eid)
    mod_label = "modified: " + aug_key_rand.split("_")[-1]


    # Set the plot layout:
    fig, ax = plt.subplots(1, 3, figsize=(15, 6), constrained_layout=True)
    fig.suptitle("EID:" + chosen_eid)

    for num in range(100, 103):

        plt.subplot(1, 3, num-99)
        plt.plot(df_norm[idx[num]:idx[num+1]]["m/z"],
                 df_norm[idx[num]:idx[num+1]].iloc[:,-1], "-b", label="original")
        plt.plot(df_norm[idx[num]:idx[num+1]]["m/z"],
                 df_aug[idx[num]:idx[num+1]].iloc[:,-1], "-r", label=mod_label)    
        plt.legend(loc="upper right")
        plt.grid()
        plt.xlim((10,150))
        plt.ylim((0,1))
        plt.title("Spectra: " + str(num))

        
def get_sample_df_from_pds_datacube(unique_eid_df):

    # This function takes the data frame corresponding to a unique EID from the data cube and transforms it to the data frame format
    # required for the data augmentation functions.

    # Initialize a loop counter:
    loop_counter = 0
    
    # make spectra index 
    spectra_idx = np.ones((len(unique_eid_df),), dtype=int)

    # Initialize empty arrays for concatenation:
    time_array = np.array([])
    temp_array = np.array([])
    m_over_z_array = np.array([])
    abundance_array = np.array([])

    for i in range (len(unique_eid_df)):

        # Assign the columns to dummy variables:
        dummy_time = np.linspace(unique_eid_df['Data'].iloc[i]['time'][0], unique_eid_df['Data'].iloc[i]['time'][-1], \
                                 len((unique_eid_df['Data'].iloc[i]['counts']) ))
        dummy_temp = np.linspace(unique_eid_df['Data'].iloc[i]['pryro_temp'][0], unique_eid_df['Data'].iloc[i]['pryro_temp'][-1], \
                                 len((unique_eid_df['Data'].iloc[i]['counts']) ))
        dummy_m_over_z = unique_eid_df['Data'].iloc[i]['amu']
        dummy_abundance_array = unique_eid_df['Data'].iloc[i]['counts']

        # Print the length for clarification:
        # print([len(dummy_time), len(dummy_temp), len(dummy_m_over_z), len(dummy_abundance_array)])

        # Vertically concatenate the 4 columns:
        time_array = np.concatenate((time_array, dummy_time))
        temp_array = np.concatenate((temp_array, dummy_temp))
        m_over_z_array = np.concatenate((m_over_z_array, dummy_m_over_z))
        abundance_array = np.concatenate((abundance_array, dummy_abundance_array))
        
        # An index to extract spectra:
        spectra_idx[i] = (len(time_array))

        # Increment the loop counter variable:
        loop_counter += 1

    # # Display the loop counter value:    
    # print(loop_counter)

    # Initialize an empty data frame:
    df_per_sample = pd.DataFrame()

    # Assign the vertically concatenated columns to the data frame columns:
    df_per_sample['time'] = time_array
    df_per_sample['temp'] = temp_array
    df_per_sample['m/z'] = m_over_z_array
    df_per_sample['abundance'] = abundance_array

    # Return the data frame:
    return (df_per_sample, spectra_idx)        

In [4]:
# Change to the home directory:
os.chdir('/home/jupyter')

# Loading the Arushi corrected version:
master_pds_df = pd.read_hdf('EGAMS_PDS_Data_AMU;10-150_OverallNorm;Final.h5')

In [5]:
# Set the main folder path of the data base to save dataframes:
main_folder_path = '/home/jupyter/samurai_data_base/'

# Define the parameters of the 6 base data augmentations:
base_aug_param = {
    # "degree_shift": 1,
    "degree_shift": 2,
    "percent_random_peaks_to_shift": 50,
    "percentage_intensity": 30,
    "arushi_noise_level": 0.05,
    "value_stretch_param": 5
}

# Find the number of unique sample EIDs
unique_eid = master_pds_df['Sample ID'].unique()
print(unique_eid)

for eid_num in range(0, len(unique_eid)):
# for eid_num in range(0, 2):    

    # Get all the data points of a given sample:
    indx = np.where(master_pds_df['Sample ID'] == unique_eid[eid_num])
    unique_eid_df = master_pds_df.iloc[indx]
    print(len(unique_eid_df))

    # Obtain the necessary data frame from the data cube:
    df_per_sample, idx = get_sample_df_from_pds_datacube(unique_eid_df)
    
    # Remove the background noise:
    df_ex_noHe_noBack = remove_background_abundance_pds(df_per_sample)
    
    # Apply the 6 base data augmentation and finally normalize:
    idx_total = np.insert(idx, 0, 0)
    df_ex_noHe_noBack_aug = bda_pds_each_spectra_final_norm(df_ex_noHe_noBack, base_aug_param, idx_total)
    
    # Insert the EID at the beginning of the dataframe:
    # (Ref: https://builtin.com/data-science/pandas-add-column)
    df_ex_noHe_noBack_aug.insert(0, "EID", str(unique_eid[eid_num]) + '.csv')
    
    # Obtain the combinatorial data augmentations:
    df_coll = get_comb_data_aug(df_ex_noHe_noBack_aug, final_comb_aug_type)  
    
    # # Display the data augmentations:
    # display_data_aug_pds(df_coll, idx_total)
    
    for key_main in df_coll:

        # Obtain the folder name:
        folder_last_name = key_main.split("_")[-1]
        folder_name = main_folder_path + folder_last_name
        # print(folder_name)
        # print(key_main)

        # Change to the suitable directory:
        os.chdir(folder_name)

        # Extract the DataFrame of the current key:
        chosen_df = df_coll[key_main]

        # Save the dataframes:
        filename = key_main.split("_")[1] + '.hdf'
        chosen_df.to_hdf(filename, key='data')    

[25048 25257 25297 25306 25327 25333 25350 25366 25484 25493 25500 25505
 25515 25530 25538 25565 25572 25579 25596 25635 25644 25653 25660 25669
 25695 25705 25719]
1332
895
896
897
829
827
827
830
830
829
837
838
771
832
770
832
770
770
736
737
787
787
787
796
773
773
787
